In [1]:
# to avoid to restart kernel when external modules are modified
%load_ext autoreload
%autoreload 2

In [2]:
import os

images_path = "/Volumes/Extreme SSD/astro/noches-estrelladas/2025/2025-agosto-19/subs"


all_img_files = [filename for filename in os.listdir(images_path) if not filename.startswith(".")]

In [3]:
all_img_files[:1]

['DSC01221.JPG']

In [6]:
from PIL import Image


def resize_img(img: Image.Image, max_size: int) -> Image:
    """Resize a PIL Image so its longest side is exactly max_size, preserving aspect ratio.
    This will upscale images smaller than max_size and downscale images larger than max_size.
    """
    width, height = img.size
    # Compute scaling factor so that the longest side becomes max_size
    scale = max_size / float(max(width, height))
    new_size = (int(round(width * scale)), int(round(height * scale)))

    # Use high-quality resampling filter
    return img.resize(new_size, Image.Resampling.LANCZOS)


output_dataset_path = "/Volumes/Extreme SSD/astro/noches-estrelladas/2025/2025-agosto-19/subs-hd"
format_img_filename = lambda filename : f"{filename.split('.')[0]}.jpg"


for img_filename in all_img_files:
    filepath = os.path.join(images_path, img_filename)

    with Image.open(filepath) as img:
        img = resize_img(img, max_size=1920)

        img.save(
            os.path.join(output_dataset_path, format_img_filename(img_filename)),
            format="JPEG"
        )

In [ ]:



with Image.open(os.path.join(images_path, all_img_files[0])) as img:
    HEIGHT = img.height
    WIDTH = img.width


HEIGHT, WIDTH

(4000, 6000)

In [ ]:
import numpy as np


def get_base_img(width: int, height: int = 600) -> Image:
    """Creates an empty black image with the (height, width) size"""
    return np.zeros((height, width, 3), dtype=np.uint8)

In [11]:
def screen_blend(base: np.array, top: np.array) -> np.array:
    MAX_INTENSITY = 255
    return MAX_INTENSITY - ((MAX_INTENSITY - base) * (MAX_INTENSITY - top) // MAX_INTENSITY)

def lighten_blend(base: np.array, top: np.array, comet_decay: float = 0.95) -> np.array:
    faded = (base.astype(np.float32) * comet_decay).clip(0, 255).astype(np.uint8)
    return np.maximum(faded, top)

def reduce_bright(img_arr: np.array, amount: int = 5) -> np.array:
    # Convertir a int16 para evitar underflow, luego clip y regresar a uint8
    img_int = img_arr.astype(np.int16) - amount
    return np.clip(img_int, 0, 255).astype(np.uint8)

In [16]:
from tqdm import tqdm


star_trails_img = get_base_img(WIDTH, HEIGHT)

all_img_files.sort()

for idx, filename in enumerate(tqdm(all_img_files)):
    img_filepath = os.path.join(images_path, filename)
    img = np.array(Image.open(img_filepath), dtype=np.uint8)

    #reduced_bright = reduce_bright(img)

    # Mezcla vectorizada (sin bucles!)
    star_trails_img = lighten_blend(star_trails_img, img, 0.999999)

    frame = Image.fromarray(star_trails_img)
    frame.save(os.path.join("output", "frames", f"000{idx+1}_frame.jpg"), format="JPEG")

100%|██████████| 250/250 [01:28<00:00,  2.81it/s]


In [ ]:
from IPython.display import display


result = Image.fromarray(star_trails_img)
display(result)